# Embeddings

Привет! Сегодня ты поработаешь с эмбеддингами: сделаешь классификатор эмоции твитов. Для начала, загрузи их:

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [ ]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Заимпортируй библиотеки и сделай работу скриптов вопсроизводимой.

In [ ]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

In [ ]:
len(data)

In [ ]:
# data = data[:10000]

Посмотрим на данные

In [ ]:
data.head()

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

Текст очень грязные. Надо добавить очистку текста в его предобработку. 

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Стокенизируем текст, избавим от знаков пунктуации и мелких слов.

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

Загрузим предобученную модель эмбеддингов. Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data . 

In [ ]:
word2vec = api.load("word2vec-google-news-300")

In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

Эмбеддинги не нормализированны, поэтому это тоже надо делать (нейросети и не только любят нормальные данные).

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(self.word2vec.vectors, axis=0)
        self.std = np.std(self.word2vec.vectors, axis=0)

    def __getitem__(self, item):
        # print(self.data[self.feature_column])
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        # print('get_tokens_', text)
        tokens = self.get_tokens_(text)
        # print('get_embeddings_', tokens)
        embeddings = self.get_embeddings_(tokens)
        # print('return')
        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        line = tokenizer.tokenize(text.lower())
        filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]

        return filtered_line

    def get_embeddings_(self, tokens):

        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec and len(w) > 3]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

## Average embedding
---
Попробуем получить векторное представление предложения из эмбеддингов слов. Самый простой вариант: усреднить вектора по всем словам. Полученный вектор можно отправить любому классификатору как вектор признаков.

Посмотрим, насколько хорошо усреднее работает для определение эмоций твитов. Сделаем их визуализацию.

In [ ]:
[print(len(dev))]

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]
print(len(example_indexes))

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Можно получить более аккуратными алгоритмами, но данный алгоритм покажет сложность задачи и поможет оценить требования к классификатору.

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples['features']) # Обучи PCA на эмбеддингах слов

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Скорее всего, ты увидел не сильно различающиеся классы. Возможно, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую(хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 0

def average_emb(batch):
    # print(batch)
    # print(batch[0])
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
for batch_idx, data in enumerate(train_loader, 0):
    x, y = data
    print(data['features'].size())
    print(data['targets'])
    break

Для обученния и тестирования нейросетевой модели сделаем отдельные функции.

In [ ]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    print('training')
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    # print('model.train()')
    for batch in pbar:
        # print('batch')
        # print(batch["features"])

        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        # print(1111)
        # print('preds')
        # Получи предсказания модели
        preds = model(features)
        optimizer.zero_grad()

        # print('preds', preds)
        # print('targets', targets)
        # print(preds.size(), targets.size())
        loss = criterion(preds, targets) # Посчитай лосс
        # print('loss', loss)
        # Обнови параметры модели
        loss.backward()
        # print('backward')
        optimizer.step()
        # print('optimizer')
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            preds = model(features) # Посчитай лосс
            # print('preds', preds.size())
            # print('preds', preds)
            # print(np.argmax(preds, axis=1))
            # preds = torch.argmax(preds, dim=1)
            # # preds = torch.reshape(predsind, (-1,1))
            # print('preds', preds)
            # print('targets', targets)
            loss = criterion(preds, targets) # Посчитай точность модели
            # print('loss', loss)

            
            acc_temp = (preds[:,1] > preds[:,0]).int()
            acc = (acc_temp == targets).sum().item()/len(targets)

            mean_loss += loss.item() # The item() method extracts the loss’s value as a Python float.
            mean_acc += acc

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

In [ ]:
x = torch.FloatTensor([[0.2, 0.1, 0.7],
                       [0.6, 0.2, 0.2],
                       [0.1, 0.8, 0.1]])

y = torch.argmax(x, dim=1)
print(torch.reshape(y, (-1,1)))
# print(x == y.values)
# x[y]

Создадим модель, оптимизатор и целевую функцию. Можете создавать любую модель, с любым количеством слоев, функций активации и прочее.


In [ ]:
import torch.nn as nn
from torch.optim import Adam

class TwoLayerNet(nn.Module):
  def __init__(self, D_in, H, D_out):
    super(TwoLayerNet, self).__init__()
    self.linear1 = nn.Linear(D_in,H)
    self.linear2 = nn.Linear(H,D_out)
    self.softmax = nn.Softmax()
  def forward(self,x):
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    preds = self.softmax(y_pred)
    return preds

# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
# print(vector_size) # 300
num_classes = 2
lr = 1e-2
num_epochs = 2

device = "cuda"
model = TwoLayerNet(vector_size, 20, num_classes).to(device)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=lr)

Обучим модель и протестируем её.
После каждой эпохи, проверим качество модели по валидационной части датасета. Если метрика стала лучше, будем сохранять модель. Подумай, какая метрика (точность или лосс) будет лучше работать в этой задаче? 

Здесь и далее реализованно с помощью лосс. Если думаешь, что лучше сравнивать модель через качество, то поменяй код выбора модели.

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

## TF-iDF
---

Вместо обычного усреднения эмбеддингов их можно дополнительно перевзвесить. Для этого воспользуемся алгоритмом `TD-iDF`. Он уже реализован в библиотеке `scikit-learn`, остается только его добавить в наш пайплайн.

In [ ]:
from collections import defaultdict
from typing import Dict

from sklearn.feature_extraction.text import TfidfVectorizer


class TwitterDatasetTfIdf(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None):
        super().__init__(data, feature_column, target_column, word2vec)

        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights

    def get_embeddings_(self, tokens):
        # print('tokens', tokens)
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights.get(token, 1) for token in tokens if token in self.word2vec and len(token) > 3]
        # print('embeddings', embeddings)
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tf_idf_(self):
        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        data_list = self.data[self.feature_column].values.tolist() # Получаем колону текст из каждой строки всего тексту итого массив массивов
        filt_list = []
        for sent in data_list:
            # print('sent', sent)
            sent_list = ""
            token_list = self.get_tokens_(sent)
            for tk in token_list:
                # print('tk', tk)
                if tk in self.word2vec:
                    sent_list += tk + ' '
            filt_list.append(sent_list)
            # print(filt_list)
            # break

        tf_idf = TfidfVectorizer()
        # print(filt_list[:5])

        # Обучи tf-idf
        tf_idf.fit_transform(filt_list)
        # print(dict(zip(tf_idf.get_feature_names(), tf_idf.idf_)))
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))


In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::100000]
print(example_indexes)

In [ ]:
dev = TwitterDatasetTfIdf(dev_data, "text", "emotion", word2vec)

Посмотрим на сложность получившейся задачи используя визуализацию через `PCA`.

In [ ]:
examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}

print(len(examples["features"]))

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples['features'])

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Создать нейросетку, обучим её на этих данных.

In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim import Adam

batch_size = 1024
num_workers = 0

train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
class TwoLayerNet(nn.Module):
  def __init__(self, D_in, H, D_out):
    super(TwoLayerNet, self).__init__()
    self.linear1 = nn.Linear(D_in,H)
    self.linear2 = nn.Linear(H,D_out)
    self.softmax = nn.Softmax()
  def forward(self,x):
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    preds = self.softmax(y_pred)
    return preds

vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
model = TwoLayerNet(vector_size,40,num_classes).to(device)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=lr)

In [ ]:
num_epochs = 2

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    # print(log)
    print(testing(model, criterion, valid_loader, device))
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

In [ ]:
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Есть ли разница в качестве между способами? Получилось ли улучшить качество модели?

----
Разница в качестве есть и возможно это зависит от кол-во текста в обучающей выборке т.к. какие то способы работаю лучше когда есть меньше данных, а какие то лучше когда есть больше данных. Как говорилось на лекции.
У меня tf-idf не дал прироста в качестве.

----

Сделай небольшое исследование:
- Попробуй сделать несколько нейросеток в качестве классификатора
- Попробуй другие предобученные эмбеддинги
- Попробуй очистить текст от ников ("@username"), url-ов и других символов

Для реализации последнего тебе могут помочь регулярные выражения (`import re`). Напише ниже отчет, что ты попробовал и что получилось.

---

Я попробовал несколько нейроннок с разным кол-во слоев но т.к. они очень много ресурсов на colab'e занимают остановился на стандартной с одним скрытым слоем.
Убрать ники с помощью регэкспа не успел...

---